
---
### **Generative Adversarial Networks for Lyrics Generation**
Raghad Taleb, ID: 210885240

in this project, we will test the RNN neural netowrk and markovian model (https://github.com/jsvine/markovify) to generate lyrics songs using data scraped from AZLyrics.


# Github Repo: https://github.com/raghadt/Generating-Lyrics
------

## Logistics Code:

In [1]:
#@title Imports { vertical-output: true , form-width: "30%" }

import tensorflow as tf
import numpy as np
import random
import os
import json

device_name = tf.test.gpu_device_name()
print('Found GPU at: {}'.format(device_name))

print(tf.__version__)

# from google.colab import drive
# drive.mount('/content/gdrive')


Found GPU at: 
2.8.0


# ⚠ : IMPORTANT: 

Please download the scraped data named "LDR_lyrics.json" by using :

```
!wget https://raw.githubusercontent.com/raghadt/Generating-Lyrics/master/LDR_lyrics.json
```

or use the file included in the zip file of the submission folder.

or you can run the code yourself, but it's going to take ~3 hours to scrape the songs :) 

**Importing Data**

1- set the path of where your data is

## Data Handling
- Data Wrangling Code - From AZLYRICS
- Data Cleaning

In [2]:
# # @title Data Wrangling Code - From AZLYRICS

# from bs4 import BeautifulSoup
# import requests
# import time

# # You can change the URL to another artist. Mine was Lana Del Rey
# BASE_URL = 'https://www.azlyrics.com/l/lanadelrey.html'
# r = requests.get(BASE_URL)
# soup = BeautifulSoup(r.text, 'html')

# # ============ Get Album names =============
# lana_dict={}
# lana_dict['album'] = {}
# # get albums and store them in lana_dict
# albums_name = soup.find_all("div", {"class":"album"})

# for album in albums_name:
#     try:
#         album_str = (album.select_one('b').get_text().strip('"'))
#         lana_dict[album_str]= {}
#     except:
#         continue;
        

    
# def get_song_page(url):
#     # url = self.get_song_url()
#     response = requests.get(url)
#     return response.content

# # =========== get song url ===================
# songs_url = []
# songs = soup.find_all("div", {"class":"listalbum-item"})
# for song in songs:
#     try:
#         songs_url.append('https://www.azlyrics.com'+song.select_one("a").get('href'))
#     except:
#         print(song)

# print("Done Creating songs url list")

# # ======= fetch the lyrics from the urls ======

# HTML_TAGS = ['br','div', 'i']
# lyrics_dictionary = {}
# # def get_lyrics(self):
# for song_url in songs_url:
#     try:
#         song_title = song_url.split('/')[-1].replace('.html','')
#         soup = BeautifulSoup(get_song_page(song_url), 'html')
#         page_lyric = soup.find_all("div", limit=21)[-1]  # since the lyrics start on 22nd div
#         lyrics = ''.join(page_lyric.find_all(text=True))

#         lyrics = [word for word in lyrics.split(' ') if word not in HTML_TAGS]

#         song_lyrics = (" ".join(lyrics[19:])) #remove the warning

#         lyrics_dictionary[song_title] = song_lyrics
#         # break
#     except:
#         print(song_url)
#     time.sleep(10) # stop for 10 seconds so it doesn't request too much





**Download Dataset**

In [3]:
!wget https://raw.githubusercontent.com/raghadt/Generating-Lyrics/master/LDR_lyrics.json

--2022-04-29 19:36:01--  https://raw.githubusercontent.com/raghadt/Generating-Lyrics/master/LDR_lyrics.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 547793 (535K) [text/plain]
Saving to: ‘LDR_lyrics.json’

LDR_lyrics.json     100%[===================>] 534.95K  --.-KB/s    in 0.04s   

2022-04-29 19:36:02 (13.2 MB/s) - ‘LDR_lyrics.json’ saved [547793/547793]



In [4]:
#@title Importing Data { form-width: "30%" }

# data_path = '/content/LDR_lyrics.json'  #@param {type: "string"}

# Open the data in the path
with open('LDR_lyrics.json', 'r') as f:
  lana_lyrics = json.load(f)

# Join the the songs lyrics into one string value
lyrics = ""
lyrics_array=[]
for song in lana_lyrics:
  lyrics+= lana_lyrics[song] + '\n'
  # lyrics_array.append(lana_lyrics[i])
  


lana_lyrics: 
ultraviolence: '....',
borntodie: '....'



In [5]:
def exclude_char(st,execluded_chars):
  clean_st = ''
  for char in st:
    if char not in execluded_chars:
      clean_st+=char
  return clean_st

In [6]:
#@title Data Cleaning { form-width: "30%" }


# text = text.replace('(','').replace(')','')
execluded_chars = '/!()*.:=[]«»;؛,،~?؟#\u200f\ufeff\“\"\'\”'

text = ""


#Here, we are taking only unique sentences from each song. 
# the artist can have themes in their music, therefore it's not fair to take the unique sentences from the whole lyrics corpus
lyrics_array_sentence = []
#loop through the lyrics dictoinary
for song_name in lana_lyrics:
  song_sentences = []
  #split the song into lines. loop through each line in the song. 
  for song_line in (lana_lyrics[song_name].split('\n')):
    song_line = exclude_char(song_line,execluded_chars)
    song_sentences.append(song_line)
  #get the unique line of each song
  song_sentences = list(set(song_sentences))
  #add them set of lines into a big list
  lyrics_array_sentence.extend(song_sentences)

# create a character set
for sentence in lyrics_array_sentence:
  for char in sentence:
    text += char
  text+='\n'

text = text.replace("\r\n","\n")
text = text.replace("\t\t\t", "\t")
text = text.replace("\r\r\n", "\n")
text = text.replace("\t\n", "\n")

#get the number of characters in a the lyrics
char_set = sorted(set(text))
print ('Number of unique characters: {} '.format(len(char_set)))

Number of unique characters: 77 


In [7]:
# char_set

**Vectorize the text**

Strings must be mapped to numerical representations. Two lookup tables are required: one to map characters to numbers, and the other to map numbers to characters.


In [8]:
# create the 2 data structures, from characters to indices
char2idx = {u:i for i, u in enumerate(char_set)}
idx2char = np.array(char_set)

In [9]:
#@title Dataset parameters { form-width: "30%" }


# batch size, default: 128
BATCH_SIZE = 128  # @param {type: "integer"}

# buffer size is used to shuffle the dataset. it will attempt to shuffle the complete sqeuence in the memory, however, it will maintain a buffer that shuffles the elements
BUFFER_SIZE = 10000  # @param {type: "integer"}

# number of RNN units #1024
N_RNN_UNITS = 256   # @param {type: "integer"}

# length of text chunks for training, default 100
MAX_LENGTH = 50  # @param {type: "integer"}

# the embedding dimensions256
EMBEDDING_DIM =  2000   # @param {type: "integer"}

# the length of the vocabulary in chars
VOCAB_SIZE = len(char_set)

print("Batch size: {} \
      \nBuffer size: {}\
      \nNumber of RNN Units: {}\
      \nMax input length: {} \
      \nVocabulary size: {} \nEmbedding dimension: {}".format(
            BATCH_SIZE, BUFFER_SIZE, N_RNN_UNITS, MAX_LENGTH, VOCAB_SIZE, EMBEDDING_DIM
        )
)

Batch size: 128       
Buffer size: 10000      
Number of RNN Units: 256      
Max input length: 50       
Vocabulary size: 77 
Embedding dimension: 2000


**Creating Training Examples and Targets for the Model**
Now, we need to segment the text into example sequences. There are *seq_length* characters in each input sequence. The corresponding targets in each input sequence are the same length as the input sequence, except shifted one character to the right.
Example: 
- Original Text: "Ground control to major Tom"
- Input: "Ground control to major To"
- Target: "round control to major Tom"



In [10]:
#@title Obtaining input and target data


input_data = []
target_data = []

for c in range(0, len(text)-MAX_LENGTH, MAX_LENGTH):
    inps = text[c : c + MAX_LENGTH]
    tars = text[c + 1 : c + 1 + MAX_LENGTH]

    input_data.append([char2idx[i] for i in inps])
    target_data.append([char2idx[t] for t in tars])

# print (np.array(input_data).shape)
# print (np.array(target_data).shape)

Now that we have our inpnut and target data, we will need to slice our original dataset to convert the text vector into a stream of indices representing the characters.

In [11]:
#@title Batch datasets

#split the original dataset into sequences, we must shuffle and pack  the data as batches before feeding it into the model
dataset = tf.data.Dataset.from_tensor_slices((input_data, target_data)).shuffle(BUFFER_SIZE)

# convert characters to sequences of batch size we pre-defined
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

dataset


<BatchDataset element_spec=(TensorSpec(shape=(128, 50), dtype=tf.int32, name=None), TensorSpec(shape=(128, 50), dtype=tf.int32, name=None))>

-------
# Building the Network

The model will be trained to try and predict "What is most likely to follow a certain character, or a sequence of characters? ". This way the model can generate text depending on the style of the data it is given.

We train the model to predict the sequence of characters we preprocessed already, and each time step the model will be fed a new input sequence.

Since RNNs have an internal state that is based on the previous elements, to try and predicit what is the next character, given all the characters computed so far.

To build the model, we will use  `tf.keras.Sequential` as a base for the model.
3 different layers will be used:

* `tf.keras.layers.Embedding`: The input layer. This is a trainable lookup table to map the numbers of each character to a vector with the embedding dimensions: `embedding_dim`
* `tf.keras.layers.GRU`:GRU is a type of RNN. we will use the pre-defined number of RUNN units as a size `units=rnn_units`
* `tf.keras.layers.Dense`: The output layer. it will provide outputs with `vocab_size` size.



In [12]:
#@title Set up generator network structure { form-width: "30%" }

# Define input and output around the RNN (GRU)
def build_model(vocab_size=VOCAB_SIZE, embedding_dim=EMBEDDING_DIM, n_rnn_units=N_RNN_UNITS, batch_size=BATCH_SIZE):
    # we will use tf.keras.Sequential to define the model, the following layers are used to construct the sequential model:
    model = tf.keras.Sequential([
          #Embedding layer is an input layer. This is a trainable lookup table to map the numbers of each character to a vector with the embedding dimensions;               
            tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                      batch_input_shape=[batch_size, None]),
            # LSTM is a type of RNN. we will use the pre-defined number of RUNN units as a size
            tf.keras.layers.LSTM(n_rnn_units,
                                return_sequences=True,
                                stateful=True,
                                recurrent_activation='sigmoid',
                                recurrent_initializer='glorot_uniform'
                                ),
            # finally, the  Dense layer is the output layer, with VOCAB_SIZE as outputs.
            tf.keras.layers.Dense(vocab_size)
        ])
    model.summary()
    return model

model = build_model()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (128, None, 2000)         154000    
                                                                 
 lstm (LSTM)                 (128, None, 256)          2311168   
                                                                 
 dense (Dense)               (128, None, 77)           19789     
                                                                 
Total params: 2,484,957
Trainable params: 2,484,957
Non-trainable params: 0
_________________________________________________________________


Define the parameters of the model, loss function, learning rate

In [13]:
# Define the loss function
def loss_function(labels, logits):
    # categorical cross entropy is used because it is applied across the last dimension of the predictions.
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)


# Define the optimiser
# learning rate will influence the model outcome. learning rate is a parameter that can be tuned to optimize the model, 
# and it represents how big of a step the model is taking. We can change learning rate from the parameters of the model.
learning_rate = 0.001  #@param{type:"raw"}

#We will use Adam optimizer. Although Adam is generally considered to be relatively robust to the choice of hyperparameters.
# it is sometimes necessary to change the learning rate from the default value.
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.5)

# Compile the model
model.compile(optimizer, loss_function)


The output distribution of the model needs to be sampled to get actual character indices. The logits of the vocabulary describe the output distribution. To avoid getting stuck in a loop, it is important to select samples from this distribution. Taking the argmax of the distribution can easily cause the model to become stuck.


In [14]:
#@title Test configuration with one example { form-width: "30%" }

for input_example_batch, target_example_batch in dataset.take(1):
    # Run the batch through the model
    example_batch_predictions = model(input_example_batch)

    # sample over the output distribution to get an actual prediction from the model.
    # The distribution is defined by the logits over the characters vocabs
    sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
    sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy() 
    
    # Double check the shape of the output
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

    # Decode the indices to see the text predicted by the (untrained) model
    print("Input: \n", repr("".join(idx2char[input_example_batch[0]])), "\n")
    print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices])))

(128, 50, 77) # (batch_size, sequence_length, vocab_size)
Input: 
 'danger\nFor the sixth page\nI can tell I can tell\nMo' 

Next Char Predictions: 
 'lABt7vNQ5órxàolUTjô1œ9Vr4tocz  zdfWNmsHîp70ROUkvz–'


## Training The Model:

In [ ]:
# #@title Save checkpoints during training { form-width: "30%" }

# from google.colab import drive
# drive.mount('/content/gdrive')

# # Directory where the checkpoints will be saved
# path = 'My Drive/CC_CW_v3/checkpoint/' #@param{type: 'string'}
# full_path = "/content/gdrive/" + path + "ckpt_{epoch}" 

# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#                       filepath=full_path,
#                       save_weights_only=True)

In [ ]:
#@title Train the model { form-width: "30%" }

# default: 10
n_epochs =  100# @param{type: "integer"} 
# history = model.fit(dataset, epochs=n_epochs, callbacks=[checkpoint_callback])

In [ ]:
# from plot_keras_history import plot_history
# import matplotlib.pyplot as plt
# plot_history(history.history, path="standard.png")
# plt.show()

-----
## Generate Text
For the text generation part, we will use a batch size of one for this prediction step to keep things simple since a fixed batch size is required once the model has been built due to the way RNN state is passed from timestep to timestep. We will need to restore the model that we already trained and saved, rebuild it and restore weights from the checkpoints. 

As a start, the model selects a start string based on the sent parameter, then initializes the RNN state, and sets the number of characters to generate. we will use the start string and the RNN state to determine the prediction distribution of the next character.

Following that, we will calculate the predicted character's index using a multinomial distribution. We will use the predicted character as our next input.

RNN state is returned by the model so more context can be added to the model instead of only one word. 
The model learns as it gets more context by seeing the previous predicted words and by providing the modified RNN states again. The model picks up on details like capitalization, paragraphs and emphases through the generated text.




In [15]:
#@title Set up text generation function { form-width: "30%" }

def generate_text(model, input_data, n_characters_output=1000):
    # First, vectorize the input text as before
    input_eval = [char2idx[s] for s in input_data]
    input_eval = tf.expand_dims(input_eval, 0)

    # We'll store results in this variable
    text_generated = []

    # Generate the number of characters desired
    model.reset_states()
    for i in range(n_characters_output):
        # Run input through model
        predictions = model(input_eval)

        # Remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # Using a categorical distribution to predict the character returned by the model
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        # Add the predicted character to the output
        text_generated.append(idx2char[predicted_id])

    # Return output
    return (input_data + ''.join(text_generated))

In [16]:
# # download checkpoints
# !wget https://drive.google.com/uc?id=1eBR2RHOmaNUV93G5qb13EHY9LygEbUi2
!pip install gdown
!gdown --id 1eIaMWKHaF7uswUDzlWXIi7ChRsaij9aO


/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1eIaMWKHaF7uswUDzlWXIi7ChRsaij9aO
To: /content/checkpoint_100.zip
100% 27.7M/27.7M [00:00<00:00, 74.3MB/s]


In [17]:
!unzip checkpoint_100.zip

Archive:  checkpoint_100.zip
   creating: checkpoint_100/
  inflating: __MACOSX/._checkpoint_100  
  inflating: checkpoint_100/ckpt_100.index  
  inflating: __MACOSX/checkpoint_100/._ckpt_100.index  
  inflating: checkpoint_100/ckpt_100.data-00000-of-00001  
  inflating: __MACOSX/checkpoint_100/._ckpt_100.data-00000-of-00001  
  inflating: checkpoint_100/checkpoint  
  inflating: __MACOSX/checkpoint_100/._checkpoint  


In [18]:
!ls

checkpoint_100	checkpoint_100.zip  LDR_lyrics.json  __MACOSX  sample_data


In [19]:
#@title Restore latest checkpoint and build model { form-width: "30%" }

# path = 'My Drive/Work/Colab/TextGen/' #@param{type: 'string'}
# path = 'My Drive/CC_CW_v3/checkpoint_100/' #@param{type: 'string'}
batch_size = 1

model = build_model(batch_size=batch_size)
# model.load_weights(tf.train.latest_checkpoint("/content/gdrive/" + path))
model.load_weights(tf.train.latest_checkpoint("checkpoint_100"))


model.build(tf.TensorShape([batch_size, None]))

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 2000)           154000    
                                                                 
 lstm_1 (LSTM)               (1, None, 256)            2311168   
                                                                 
 dense_1 (Dense)             (1, None, 77)             19789     
                                                                 
Total params: 2,484,957
Trainable params: 2,484,957
Non-trainable params: 0
_________________________________________________________________


In [20]:
#@title Generate text! { form-width: "30%" }

input_data = "happy "  # @param {type: "string"}
n_characters_output = 1000 #@param 
# print(generate_text(model, input_data=input_data, n_characters_output=n_characters_output))
print(generate_text(model, input_data,n_characters_output))


happy bars
What happick and roller
Got youre big stristenders are between from your pow
Its just a pin-yes you
Its like I never knew do
To runs we have you

Soul not sad

Oh Oh oh oh oh oh-oh oh
Yeah oh Yeah oh
Dont you stay
Pretty dreams that
And I want is is a prize come to the eitch just for real for reason with you
Was only since you dont like you I heard and push on


Come on back to my neise
I got you I wanna go tappem
I cant Rall
Its go Lany on your head can untup baby you I like my baby bout screess Can I see to leave

Baby youre a dark so bad Nos gone Monight line but high Wailia roller
The wind Im gone cause were going
Bereaking imates Cypaties
One day is it forever
Hey boo lets go back on the promised
I can and e
Can you like fine man jive
I said Baby Youre so hot courser they
Even then I like to keep that way
Breaking up radit off me
You dont wanna diate from got that one-top of hard to sleep like smile and young youre the end them
Ooh oh
Theres no pray I sight But to pread

------
# Markovian Model

In [ ]:
!pip install markovify

     |████████████████████████████████| 235 kB 7.8 MB/s 
  Created wheel for markovify: filename=markovify-0.9.4-py3-none-any.whl size=18628 sha256=b0bc245fc832936f41688346e57dc436df7641655bdd350716c16fbef677f016
  Stored in directory: /root/.cache/pip/wheels/36/c5/82/11125c5a7dadec27ef49ac2b3a12d3b1f79ff7333c92a9b67b
Successfully built markovify


In [ ]:
import markovify

In [ ]:
# len(set(all_lines))

In [ ]:
all_lines = list(set(lyrics_array_sentence))
big_poem = "\n".join([line for line in random.sample(all_lines, len(all_lines))])


In [ ]:
model = markovify.NewlineText(big_poem)

In [ ]:
for i in range(14):
    print(model.make_sentence())

I didnt believe you when we booed you off easy its alright
Its just that good
Pin up girls at the movie stars
But you the king of the scale of the sea hands all over now
How about about the things youve done to me everything you got so scared I thought
Be proud come out with a song
The Queen of Saigon
But in my hair up
I still love me youll love me hardcore then dont walk away
Did you want a little song
You know Im wrong
The kind of people you meet on the run with you my king
Crying cause I like it when its me
Mary swaying softly to her bare ass


# Comparing Results
Assumption: generated text will most likly contain the most repeatitive words of the data corpus.

we will remove stop words since they're most like to be repeated

In [ ]:
import re

In [ ]:
word = 'summer'
searched_lines = sorted(
    [line for line in all_lines if re.search(r"\bsummer\b\s\w", line)], 
    key=lambda line: line[re.search(r"\bsummer\b\s", line).end():]) 

for line in searched_lines[0:10]:
    print(line)

I had bright wishes in the summer I was bathing in sunlight
Its the middle of winter but in my mind its summer Mike
High tops in the summer Summer what What What
High tops in the summer Top out hop out hop out
Hip-hop in the summer What What What
Hot summer and cold watermelon
Hot like the summer and mean like a child
Hear the birds on the summer breeze
Like a summer breeze
I hear the birds on the summer breeze


In [ ]:
from collections import Counter
from nltk import word_tokenize
from nltk.corpus import stopwords
import nltk
import string

In [ ]:
nltk.download('stopwords')
#define stop words, using set to get a unique set.
stop = set(stopwords.words('english') + list(string.punctuation))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Finding most common words**

In [ ]:
# words_list = text.replace('\n',' ').split(' ')
words_list = []
for word in text.replace('\n',' ').split(' '):
  if (word.lower() not in stop):
    words_list.append(word.lower())

In [ ]:
Counter(words_list).most_common(30)

[('im', 823),
 ('like', 794),
 ('', 673),
 ('baby', 494),
 ('dont', 493),
 ('know', 437),
 ('love', 437),
 ('youre', 429),
 ('got', 309),
 ('get', 293),
 ('want', 265),
 ('say', 235),
 ('oh', 232),
 ('cause', 227),
 ('wanna', 218),
 ('one', 212),
 ('go', 209),
 ('never', 203),
 ('come', 197),
 ('way', 189),
 ('make', 173),
 ('yeah', 172),
 ('think', 170),
 ('life', 164),
 ('take', 157),
 ('see', 153),
 ('time', 152),
 ('cant', 151),
 ('back', 147),
 ('said', 147)]

we observe that **love, think, life, time** are very repeative, now let's see if they are repetitive as well in our text generated text
NOTE: I took the text that was used for evaluting the model. that's why it's hard coded

**RNN Model**

In [ ]:
RNN_text= "happytrawsoars that Imbana Jimmy pring Or your waver I conty up now you do it doesnt know come Used to love to Cheaver fasorio let theres no you loved morning They she doice me langer egas car wrice new scregiase We gonna say that the what do you here The world is deadd Im lonely goodbybody so You gonna learny sce and I can take it all you Swinging in my dress Learn me just you come Youll be appeat I smoke until we were born that the world But dont get you it enough Glamorou ride are run run awour hemelt me fresh that on the man yeah Hm the man when you call God knows I lose your little caoset otch Back around when he smoking now And Ilsed get her Youre lying theresome the count you should be bad Ats not to heart Oh-oh-oh-oh-h-h State his golden dangeresser To see her making me like sheeper Babe its alright Why Mayo believe in my disant streets But honest my sw He wre lying in the run Im beliou still get out laughing Dead Im doing no to lose at elseamanost daddy Least your flection Oh dont"
RNN_words_list = []
for word in RNN_text.split(' '):
  if (word.lower() not in stop):
    RNN_words_list.append(word.lower())

for i in Counter(RNN_words_list).most_common():
  # print(j)
  if i[0] == 'love' or i[0] =='life' or i[0] =='think' or i[0] =='time' or i[0] =='i':
    print(i)


('love', 1)


**Markovian Model**

In [ ]:
mark_text= "LA I dont pop my cork for every experience and an obsession for freedom that terrified me to the Beach Boys He dont like no one which means theres only truth and God is playing a little bit of paradise Keep me safe in the sky with ice tonight Im lying in the sand Anything you like you were seeing yeah You said I know thats not a school girl crush You cant be with the blue mascara You should never ever stop No Make me smile and your gun Even if youre able They call you up twice hang up the H of the Gods Smoking all my money gave you all the ways The secrets you keep it moving babe Ill do what you desire"

mark_words_list = []
for word in mark_text.split(' '):
  if (word.lower() not in stop):
    mark_words_list.append(word.lower())

for i in Counter(mark_words_list).most_common():
  # print(j)
  if i[0] == 'love' or i[0] =='life' or i[0] =='think' or i[0] =='time' or i[0] =='i':
    print(i)


## Appendix  - Resources :